In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from urllib.request import urlopen
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import requests

import re

In [112]:
movie = pd.read_csv('data_ver4.csv')

In [111]:
len(movie)

20528

# 중간 저장

# 시작

In [213]:
movie_df = pd.read_csv('data_ver4.csv')

# 1. 중복된 영화 찾기

In [215]:
pd.set_option('display.max_rows', 10000)

# 중복된 영화 확인하기
name_counts = movie_df['영화명'].value_counts()

In [216]:
# 영화 카운트만 따로 df로 만들기
n_c_df = pd.DataFrame(name_counts)

In [217]:
print(n_c_df.iloc[0, :].name)
print(int(n_c_df.iloc[0, :].values))

로미오와 줄리엣
6


In [218]:
# 중복된 영화 리스트

dupl = []

for i in range(len(n_c_df)):
    if int(n_c_df.iloc[i, :].values) > 1:
        dupl.append(n_c_df.iloc[i, :].name)

In [219]:
len(dupl)

483

In [124]:
movie_df['매출액']

0          26000
1         300000
2        1830000
3           4000
4           5000
          ...   
20523    2992500
20524      10000
20525      88000
20526    3543400
20527       4000
Name: 매출액, Length: 20528, dtype: int64

# 2. 감독 따기

In [220]:
directors = []

for i in range(0, len(movie_df)):
    tmp = str(movie_df["감독"][i]).split(',')[0]
    
    directors.append(tmp)    

pd.Series(directors).value_counts() 

# nan 값이 많네?
# movie.iloc[[14370]]['감독']
# movie['감독'] = movie['감독'].astype(str) # str로 변경

# 첫번째 감독으로 변경
movie_df['감독'] = directors

In [121]:
# # 기타 장르 삭제
# movie_df = movie_df[movie_df['장르'] != "기타"]

## 숫자데이터 데이터 타입변환하기(object->int)

In [37]:
nums = []

for num in movie_df['누적매출액']:
    nu_num = num.replace(",", "")
    nums.append(int(nu_num))
    

In [38]:
movie_df['누적매출액'] = nums

In [39]:
nums = []

for num in movie_df['매출액']:
    nu_num = num.replace(",", "")
    nums.append(int(nu_num))
    

In [40]:
movie_df['매출액'] = nums

In [41]:
nums = []

for num in movie_df['관객수']:
    nu_num = num.replace(",", "")
    nums.append(int(nu_num))
    

In [42]:
movie_df['관객수'] = nums

In [43]:
nums = []

for num in movie_df['누적관객수']:
    nu_num = num.replace(",", "")
    nums.append(int(nu_num))
    

In [44]:
movie_df['누적관객수'] = nums

In [45]:
nums = []

for num in movie_df['스크린수']:
    nu_num = num.replace(",", "")
    nums.append(int(nu_num))
    

In [46]:
movie_df['스크린수'] = nums

In [47]:
nums = []

for num in movie_df['상영횟수']:
    nu_num = num.replace(",", "")
    nums.append(int(nu_num))
    

In [48]:
movie_df['상영횟수'] = nums

In [125]:
# 누적 매출액 0 갯수 854
movie_df = movie_df[(movie_df['누적매출액'] > 1) & (movie_df['누적관객수'] > 1)]

In [434]:
open_year = list(nan_director.loc[nan_director['영화명'] == '나의 고향']['개봉일'])[0]
year = open_year.split('-')[0]
year

AttributeError: 'float' object has no attribute 'split'

# 네이버 영화 감독 정보 크롤링

In [9]:
# 링크 접속
def url_search(movie_name):
    url1 = "https://movie.naver.com/movie/search/result.naver?query={}&section=all&ie=utf8".format(movie_name)
    html1 = requests.get(url1)
    soup1 = BeautifulSoup(html1.text, 'html.parser')
    
    return soup1

In [35]:
def one_or_more(soup1):
    tag_search_child = soup1.find("div", class_ = "search_header first-child")
    if bool(tag_search_child) == True:
        num1 = soup1.find('span', class_ = 'num')
        num2 = num1.get_text()
        num3 = num2[3]
    
        return int(num3)
    else:
        return "없음"

In [36]:
soup_ = url_search('나의 고향')
type(one_or_more(soup_))

int

## 영화 검색하니 하나짜리 영화

In [95]:
# href 태그 찾기

def href_tag(soup1):

    tag_search_list1 = soup1.find("ul", class_ = "search_list_1")
    
    if bool(tag_search_list1) == True:
        tag_a = soup1.select('div#old_content > ul.search_list_1 > li > p.result_thumb > a')
        global href
        for i in tag_a:
            href = i.attrs['href']
    
        return href
    
    else:
        return "없음"
    

## 영화가 여러 개인 경우

In [12]:
def tag_dt_title(soup1):

    tag_search_list1 = soup1.find("ul", class_ = "search_list_1")
    
    if bool(tag_search_list1) == True:
        tag_li = soup1.find_all('li')
        if bool(tag_li) is True:
            for li in tag_li:
                tag_dt = soup1.find_all('dt')
   
            return tag_dt
    
    else:
        return "없음"
        
# h1 = soup.html.body.h1 

In [487]:
def tag_dd_info(soup1):

    tag_search_list1 = soup1.find("ul", class_ = "search_list_1")
    
    if bool(tag_search_list1) == True:
        tag_li = soup1.find_all('li')
        if bool(tag_li) is True:
            for li in tag_li:
                tag_dd = soup1.find_all('dd', class_ = 'etc')
                
            return tag_dd
    else:
        return "없음"

In [491]:
def dt(dt_title):
    titles = []
    
    if dt_title != "없음":
        for dt in dt_title:
            dt = str(dt.get_text())
            if (dt.find('(') > -1):
                dt = dt[:dt.find('(')-1]
                titles.append(dt)

        return titles
    
    else:
        return "없음"

In [490]:
def dd(dd_info):
    list_dd = []
    list_even_split = []
    
    if dd_info != "없음":
        for dd in dd_info:
            dd = str(dd.get_text())
            dd = dd.split('|')
            list_dd.append(dd)

        list_odd = list_dd[0::2]
#     list_even = list_dd[1::2]
    
#     for i in range(len(list_even)):
#         if len(list_even[i]) < 2:
#             direc = list_even[i][0].replace("감독 : ", "")
#             director = direc[0]
#             tmp1 = [director]
#             list_even_split.append(tmp1)
#         else:
#             direc = list_even[i][0].replace("감독 : ", "")
#             act = list_even[i][1].replace("출연 : ", "")
#             director = direc[0]
#             actors = act[0]
#             tmp2 = [director, actors]
#             list_even_split.append(tmp2)  
        return list_odd
    else:
        return "없음"

In [493]:
a = url_search('문명사회')

dt_title = tag_dt_title(a)
dd_info = tag_dd_info(a)

titles = dt(dt_title)
info= dd(dd_info)

print(info)

없음


## 페이지 찾아들어가서 파싱

In [16]:
# 해당 페이지 링크 가져오기

def find_link(href):
    
    if href != "없음":
        url2 = "https://movie.naver.com{}".format(href)
        html2 = requests.get(url2)
        soup2 = BeautifulSoup(html2.text, 'html.parser')
    
        return soup2
    
    else:
        return "없음"

In [17]:
# dd 태그 찾기

def dd_tag(soup2):
    
    if soup2 != "없음":
        find_dir = soup2.find_all('dd')
    
        return find_dir
    
    else:
        return "없음"

In [18]:
# director 찾기
def find_director(find_dir):
    
    if find_dir != "없음":
        words = []
    
        for i in find_dir:
            i = i.get_text()
            tmp = []
            tmp.append(i)
        
            for w in tmp:
                w = w.strip('\n')
                words.append(w)
                director = words[0].split(',')[0]
        
            return director
    
    else:
        return "없음"
            

## 감독값이 결측값인 데이터셋

In [230]:
nan_director = pd.read_csv('data_ver5.csv')

In [231]:
type(nan_director['개봉일'][0])

float

In [424]:
nan_director[nan_director['영화명'] == '코마']

,영화명,개봉일,매출액,누적매출액,관객수,누적관객수,스크린수,상영횟수,대표국적,배급사,등급,장르,감독,배우
3554,코마,2020-11-12,18739450,18739450,2563,2563,46,274,러시아,(주)까멜리아이엔티,15세이상관람가,SF,NaN,NaN


In [512]:
nan_movies = list(nan_director['영화명'])
nan_movies[20:30]

['E-19 바이러스',
 '겟 아웃 맨',
 '더 콜러',
 '디 아이즈',
 '디시젼: 리퀴데이션',
 '로스트: 직소 퍼즐',
 '마린: 배틀 엔젤',
 '배틀 트랩',
 '뱅크 잡: 리크루트',
 '블러드 스크림']

In [322]:
len(nan_director)

3634

In [238]:
def year_(movie_name):
    open_year = nan_director.loc[nan_director['영화명'] == movie_name]['개봉일']
    year = list(open_year)[0]
    if type(year) != float:
        return year.split('-')[0]
    else:
        return None
    

    
print(year_('나의 고향'))
    

None


In [552]:
def select_year(info):
    years = []
    for i in range(len(info)):
        if len(info[i]) <= 4:
            year = info[i][-1]
            years.append(year)
    return years

In [553]:
def year_compare(searched_year, year_data):
    for i in range(len(searched_year)):
        if searched_year[i] == year_data:
            return searched_year[i]


# 실행함수

In [557]:
def directors(movie_names):
    
    directors = []
    
    for movie_name in movie_names:
        soup1 = url_search(movie_name)
        num = one_or_more(soup1)
        
        if num == 1:
            href = href_tag(soup1)
            soup2 = find_link(href)
            find_dir = dd_tag(soup2)
            director1 = find_director(find_dir)
            
            directors.append(director1)
            
        elif num == "없음":
            directors.append("없음")
        
            
        else:
            dt_title = tag_dt_title(soup1)
            dd_info = tag_dd_info(soup1)
            
            titles = dt(dt_title)
            info = dd(dd_info)
            
            year_data = year_(movie_name)
            searched_year = select_year(info)
            
            year = year_compare(searched_year, year_data)
                    
            if year == None:
                directors.append("없음")
            else:
                soup3 = url_search(movie_name + year)
                href2 = href_tag(soup3)
                soup4 = find_link(href2)
                find_dir = dd_tag(soup4)
                director2 = find_director(find_dir)
                
                directors.append(director2)        
                        

                        
    return directors


In [558]:
movie_names = ['나의 고향', '디스턴스', '문명사회', '바다의 법',
           '비', '스노보드', '어디에나 있는 앨리스', '언더라이프',
           '유행', '이야기']
d = ['E-19 바이러스',
 '겟 아웃 맨',
 '더 콜러',
 '디 아이즈',
 '디시젼: 리퀴데이션',
 '로스트: 직소 퍼즐',
 '마린: 배틀 엔젤',
 '배틀 트랩',
 '뱅크 잡: 리크루트',
 '블러드 스크림']


directors(['코마'])

['없음']

In [556]:
# directors(['비'])

# ['나의 고향',
#  '디스턴스',
#  '문명사회',
#  '바다의 법',
#  '비',
#  '스노보드',
#  '어디에나 있는 앨리스',
#  '언더라이프',
#  '유행',
#  '이야기']